In [17]:
#PARAMETERS
testDataFile="/Users/eliotpbrenner/PycharmProjects/SparsityBoost/data/synthetic_examples/experiments/0/alarm1000.dat"
numTestVariable=37
numTestDataPts=1000
studentJSONFile = "student.txt"
studentDataFile="student_100kSamples.csv"
studentNValsFile="student_nvals.csv"
numStudentVariable=5
numStudentDataPts=100000

100000

In [18]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [19]:
# Imports
using Base.Test
using Iterators

In [20]:
type model
  nodes::Int64
  nval::Array{Int64,1}
  dataArray::Array{Int64,2}
end

In [21]:
type jointDistWithMarginals
  pdist::Array{Float64,2}
  p_A::Array{Float64,1}
  p_B::Array{Float64,1}
end

In [22]:
# Constructor which calculates the marginals
jointDistWithMarginals(pdist::Array{Float64,2}) = jointDistWithMarginals(pdist, vec(sum(pdist,1)), vec(sum(pdist,2)))

jointDistWithMarginals

In [23]:
jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]])

jointDistWithMarginals(2x2 Array{Float64,2}:
 0.25  0.25
 0.25  0.25,[0.5,0.5],[0.5,0.5])

In [24]:
function readDataBinary(filePath::AbstractString)
  dataArray = readdlm(filePath, ' ', Int);
  nodes=size(dataArray)[2];
  nval=2*ones(Int, nodes)
  theModel = model(nodes,nval,dataArray);
end


readDataBinary (generic function with 1 method)

In [25]:
function readData(dataFilePath::AbstractString, nvalsFilePath::AbstractString)
  dataArray = readdlm(dataFilePath, ',', Int);
  nodes=size(dataArray)[2];
    nval=vec(readdlm(nvalsFilePath, ',', Int));   #2*ones(Int, nodes)
  theModel = model(nodes,nval,dataArray);
end

readData (generic function with 1 method)

In [26]:
m=readData(studentDataFile, studentNValsFile)

model(5,[2,2,3,2,2],100000x5 Array{Int64,2}:
 0  0  1  0  0
 0  0  0  1  1
 1  0  2  0  0
 0  0  2  0  0
 1  1  0  1  1
 0  0  1  0  0
 0  0  2  0  0
 0  0  0  0  1
 0  0  1  0  1
 0  0  1  0  0
 0  0  1  0  0
 0  1  0  1  1
 0  0  0  0  1
 ⋮            
 0  0  1  0  0
 1  1  2  1  0
 0  0  1  0  1
 0  0  0  0  1
 0  0  2  0  0
 0  0  1  0  0
 0  1  0  0  1
 1  1  1  1  0
 0  1  0  0  0
 1  0  1  1  0
 1  1  0  1  1
 0  0  2  0  0)

In [27]:
# test readData function
m=readData(studentDataFile, studentNValsFile)
@test size(m.dataArray) == (numStudentDataPts, numStudentVariable)

In [28]:
function verify(aJointDistributionWithMarginals::jointDistWithMarginals)
  sumA=sum(aJointDistributionWithMarginals.pdist,1)
  sumB=sum(aJointDistributionWithMarginals.pdist,2)
  @test_approx_eq(transpose(sumA),aJointDistributionWithMarginals.p_A)
  @test_approx_eq(sumB,aJointDistributionWithMarginals.p_B)
  @test_approx_eq(sum(sumA),1.0)
  return true
end

verify (generic function with 1 method)

In [29]:
goodDist=jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]], [0.5; 0.5], [0.5; 0.5])
@test verify(goodDist) == true

In [30]:
#TODO: determine why this this test fails instead of succeeding!
badDist = jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]], [0.6; 0.4], [0.5; 0.5])
@test_throws( ErrorException, verify(badDist))

ErrorException("assertion failed: |transpose(sumA) - aJointDistributionWithMarginals.p_A| <= 2.220446049250313e-12\n  transpose(sumA) = [0.5\n 0.5]\n  aJointDistributionWithMarginals.p_A = [0.6,0.4]\n  difference = 0.09999999999999998 > 2.220446049250313e-12")

In [31]:
function MI_term(i::Int, j::Int, dist::jointDistWithMarginals)
  return dist.pdist[i,j]*log(dist.pdist[i,j]/(dist.p_A[i]*dist.p_B[j])  )
end

MI_term (generic function with 1 method)

In [32]:
function MI(dist::jointDistWithMarginals)
  (a,b)=size(dist.pdist)
  return sum([(MI_term(i,j,dist)) for i=1:a, j=1:b])
end

MI (generic function with 1 method)

In [33]:
MI_term(1,1,goodDist)

0.0

In [34]:
@test_approx_eq(MI(goodDist),0)

In [35]:
t=0.01
anotherDist=jointDistWithMarginals([[0.25+t 0.25-t]; [ 0.25-t 0.25+t]], [0.5; 0.5], [0.5; 0.5])
verify(anotherDist)
@test_approx_eq(MI(anotherDist), 0.0008002134699838133)

In [36]:
testNotInDict(elt::Int, SToVal::Dict) = try
    """TODO: figure out how to propagate y up the call stack if it's not a KeyError"""
    x=SToVal[elt] 
    warn("$elt is in $SToVal with value $x")
    throw(DomainError())
  catch y
    if isa(y, KeyError)
      return true
    else
      return y
    end
  end

testNotInDict (generic function with 1 method)

In [37]:
S=Dict(1=>2, 3=>4)

Dict{Int64,Int64} with 2 entries:
  3 => 4
  1 => 2

In [38]:
testNotInDict(1,S)

DomainError()

In [39]:
function conditionalDist(m::model,i::Int,j::Int,SToVal::Dict)
  """
  TODO: that S is a dict of integers and that both are valid for the model
  TODO: vectorize the computation of the pDist
  """
  @test i!=j
  @test testNotInDict(i, SToVal) == true
  @test testNotInDict(j, SToVal) ==true
  selectedPoints = m.dataArray[reduce(&,[m.dataArray[:,condVar] .==SToVal[condVar] for condVar in keys(SToVal)]),:]
  numSelectedPoints = size(selectedPoints)[1]
  pDist = zeros(m.nval[i], m.nval[j])
  for k = 1:m.nval[i]
      for l = 1:m.nval[j]
          pDist[k,l] = size(selectedPoints[(selectedPoints[:,i] .==k-1)&(selectedPoints[:,j] .==l-1), :])[1]/numSelectedPoints
    end
  end
    jd = jointDistWithMarginals(pDist)
    verify(jd)
    return jd
end

conditionalDist (generic function with 1 method)

In [40]:
SToVal = Dict(3=>0, 4=>1)
cd = conditionalDist(m, 1, 2, SToVal)

jointDistWithMarginals(2x2 Array{Float64,2}:
 0.0344865   0.700971
 0.00431082  0.260231,[0.03879733711666485,0.9612026628833351],[0.7354578194914329,0.2645421805085671])

In [41]:
@test_throws(ErrorException,conditionalDist(m, 1, 1, SToVal))

ErrorException("test failed: 1 != 1\n in expression: i != j")

In [42]:
conditionalDist(m, 3, 1, SToVal)

LoadError: LoadError: test failed: DomainError() == true
 in expression: testNotInDict(i,SToVal) == true
while loading In[42], in expression starting on line 1

In [43]:
#TODO: determine why this test is failing
@test_throws(DomainError, conditionalDist(m, 3, 1, S))

LoadError: LoadError: test failed: ErrorException("test failed: DomainError() == true\n in expression: testNotInDict(i,SToVal) == true") was thrown instead of DomainError
 in expression: conditionalDist(m,3,1,S)
while loading In[43], in expression starting on line 2

In [44]:
conditionalDist(m, 3, 1, S)

LoadError: LoadError: test failed: DomainError() == true
 in expression: testNotInDict(i,SToVal) == true
while loading In[44], in expression starting on line 1

In [45]:
function MI(m::model,i::Int,j::Int,SToVal::Dict)
  """
  TODO: test
  """
  MI(conditionalDist(m::model,i::Int,j::Int,SToVal::Dict))
end

MI (generic function with 2 methods)

In [46]:
function MI(m::model, i::Int, j::Int, S::Array{Int64,1})
  """
  TODO: test
  """
  max=-Inf
  for p in Iterators.product([1:m.nval[i]-1 for i in S]...)
        candidate=MI(m,i,j, Dict(zip(S,p)))
    if candidate > max
      max=candidate
    end
  end
  return max
end

MI (generic function with 3 methods)

In [47]:
#MI(m,1,2, Dict(3=>1, 7=>1, 22=>1))
MI(m,1,2,Dict(3=>1, 4=>1, 5=>1))

0.1712955999170267

In [48]:
function epsilon(m::model, i::Int, j::Int, d::Int)
    min=Inf
    bestK=Array([])
    for di in (1:d)
        for k in combinations(1:m.nodes,di)
            if (i in k) || (j in k)
                continue
            end
            candidate = MI(m,i,j,k)
            if candidate < min
                min=candidate
                bestK=k
            end
        end
    end
    return bestK, min
end

epsilon (generic function with 1 method)

In [49]:
epsilon(m,1,2,1)

([5],0.16748672602478862)

In [2]:
using PyCall


In [50]:
@pyimport graphviz

In [14]:
ndModule = pyimport("libpgm.nodedata")

PyObject <module 'libpgm.nodedata' from '/Users/eliotpbrenner/anaconda/lib/python2.7/site-packages/libpgm/nodedata.pyc'>

In [15]:
ndObj=pycall(ndModule["NodeData"], PyCall.PyObject)

PyObject <libpgm.nodedata.NodeData object at 0x318fbacd0>

In [51]:
pycall(ndObj["load"],PyCall.PyObject, studentJSONFile)

PyObject None

In [53]:
gsModule = pyimport("libpgm.graphskeleton")

PyObject <module 'libpgm.graphskeleton' from '/Users/eliotpbrenner/anaconda/lib/python2.7/site-packages/libpgm/graphskeleton.pyc'>

In [55]:
gsObj=pycall(gsModule["GraphSkeleton"], PyCall.PyObject)

PyObject <libpgm.graphskeleton.GraphSkeleton object at 0x318fc4f50>

In [56]:
pycall(gsObj["load"], PyCall.PyAny, studentJSONFile )

dbnModule = pyimport("libpgm.discretebayesnetwork")
dbnObj = pycall(dbnModule["discretebayesiannetwork"], PyCall.PyAny)
adbn = pycall(dbnObj["DiscreteBayesianNetwork"], PyCall.PyObject, skel, ndb)

In [63]:
dbnModule = pyimport("libpgm.discretebayesiannetwork")


PyObject <module 'libpgm.discretebayesiannetwork' from '/Users/eliotpbrenner/anaconda/lib/python2.7/site-packages/libpgm/discretebayesiannetwork.pyc'>

In [65]:
dbnObj = pycall(dbnModule["DiscreteBayesianNetwork"], PyCall.PyAny, gsObj, ndObj)


PyObject <libpgm.discretebayesiannetwork.DiscreteBayesianNetwork object at 0x3190677d0>

In [67]:
V = pycall(dbnObj["__getattribute__"], Array, "V")

5-element Array{Any,1}:
 "Difficulty"  
 "Intelligence"
 "Grade"       
 "SAT"         
 "Letter"      

In [68]:
E = pycall(dbnObj["__getattribute__"], Array, "E")

4x2 Array{Any,2}:
 "Intelligence"  "Grade" 
 "Difficulty"    "Grade" 
 "Intelligence"  "SAT"   
 "Grade"         "Letter"

In [69]:
g = graphviz.Graph(format="svg")

In [70]:
pycall(g["node"], PyCall.PyObject, "Difficulty")
pycall(g["node"], PyCall.PyObject, "Grade")
pycall(g["edge"], PyCall.PyObject, "Difficulty", "Grade")

PyObject None

In [71]:
print(pycall(g["__getattribute__"], ASCIIString, "source"))

graph {
	Difficulty
	Grade
		Difficulty -- Grade
}

In [72]:
vertexEncodings = [v=>i for (i,v) in enumerate(V)]

Dict{Any,Int64} with 5 entries:
  "Intelligence" => 2
  "Difficulty"   => 1
  "SAT"          => 4
  "Grade"        => 3
  "Letter"       => 5

In [73]:
E

4x2 Array{Any,2}:
 "Intelligence"  "Grade" 
 "Difficulty"    "Grade" 
 "Intelligence"  "SAT"   
 "Grade"         "Letter"

In [74]:
edgesEncoded = [[vertexEncodings[E[i,1]],vertexEncodings[E[i,2]]] for i in 1:size(E)[1]]

4-element Array{Any,1}:
 [2,3]
 [1,3]
 [2,4]
 [3,5]

In [75]:
dg = graphviz.Digraph()

In [79]:
for (v,i) in vertexEncodings
    pycall(dg["node"], PyCall.PyObject, string(uppercase(Char(i+96))), v)
end

In [115]:
for rownum in 1:size(edgesEncoded)[1]
    string(uppercase(Char(edgesEncoded[rownum][1]+96)))*string(uppercase(Char(edgesEncoded[rownum][2]+96))) #*string(uppercase(Char(edgesEncoded[rownum][2]+96)))
    pycall(dg["edges"], PyCall.PyObject, 
    string(uppercase(Char(edgesEncoded[rownum][1]+96)))*string(uppercase(Char(edgesEncoded[rownum][2]+96))))
end

In [102]:
edgesEncoded[1][1]

2